In [1]:
cd ..

/home/alberto/Work/incomplete_multiview_clustering/src


In [2]:
from algorithms.concat import Concat
from algorithms.nmf import NonnegativeMatrixFactorization
from imvc.utils.utils import RandomMultiviewDataset, ConvertToPositive
from metrics.internal import InternalMetrics
from metrics.stability import StabilityMetrics
from metrics.external_validation import ExternalValidation
import warnings

In [3]:
rmd = RandomMultiviewDataset()
mvd = rmd.create_mvd(n_samples = 100, n_features = [10, 50, 100], n_clusters = [3, 4, 2])
sample_views = rmd.add_sample_views(mvd = mvd, p = [0.2, 0.3, 0.5])
imvd = rmd.add_missing_views(mvd = mvd, sample_views = sample_views)
for data in imvd:
    display(data.head())

,0,1,2,3,4,5,6,7,8,9
0,1.991881,-0.495649,-5.387850,-3.838364,7.165036,-5.399642,8.067310,1.339492,3.752539,3.047053
1,-1.885133,5.546076,-2.295305,5.176153,9.773470,-1.237217,0.562206,3.070024,9.740761,2.634843
3,-0.574464,-1.348980,-2.435940,-4.663056,5.752965,-4.797291,8.061107,1.300182,3.470789,2.724759
4,1.643816,-2.598021,-4.729443,-3.290675,6.155916,-5.381031,8.628626,1.698891,3.014388,1.983026
5,2.008131,1.556859,-5.516057,-6.157961,5.383868,-6.792920,9.857424,3.505505,3.292217,1.533625


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,5.243763,-9.781632,1.173421,11.168826,-0.209417,1.667713,5.531346,2.309046,8.889487,-1.291908,...,10.680423,-3.811889,2.641831,-9.112099,-9.790993,-2.742453,9.369780,-11.538022,6.126332,0.777960
1,6.767508,-2.753035,-8.427684,0.970619,-5.985052,2.186068,3.302715,1.644481,2.652358,3.168802,...,7.557111,-4.381570,-2.213940,-8.498212,-2.987202,5.097091,5.383278,9.720925,8.078685,4.703897
2,8.297755,-4.960467,-7.890910,2.116007,-4.280295,2.859521,4.782396,2.644120,1.678983,3.951661,...,4.769872,-2.811901,-0.787471,-7.076517,-3.758579,3.281833,4.006940,10.325980,8.832406,4.649861
4,3.396001,-9.567451,1.555135,9.834162,0.424327,0.683937,6.564827,2.084975,8.135516,-2.550616,...,10.203875,-2.296836,5.178458,-8.329568,-7.968392,-2.544517,9.747093,-7.701737,6.988804,1.276952
5,-2.721128,-8.210588,-5.904585,-8.506959,-7.766121,-4.084005,7.103536,-10.563859,-6.831833,3.549300,...,-3.486844,3.785713,-8.632021,-5.252824,7.123611,-2.507531,8.085737,-4.810181,3.464879,5.088489


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.685396,-5.415942,-9.134462,-3.146258,-10.095957,-7.679344,-1.410176,-4.410992,2.877341,0.352767,...,-2.174377,-5.002319,0.418406,7.038916,8.484965,-6.823810,4.052056,7.879174,3.964629,-5.190068
1,1.505529,-6.769119,-10.200332,-2.548948,-7.948914,-6.698631,0.677508,-2.139814,3.762669,-0.180128,...,-0.608992,-3.914612,0.213651,6.899259,6.764747,-6.462744,3.036650,9.161380,3.843180,-6.467175
2,1.304275,-6.412466,-9.546715,-3.198643,-8.802311,-5.634086,0.595770,-3.349284,2.301843,-0.513834,...,-0.076657,-4.589112,-1.515457,6.175377,8.037385,-6.608142,3.582748,8.290431,5.421841,-7.591939
3,1.682002,-4.277947,-8.618384,-3.476233,-6.792886,-7.030780,-0.425946,-3.540312,1.229474,0.214768,...,-2.217449,-4.574389,1.643387,9.417095,8.509266,-5.762628,3.278732,9.494978,2.759002,-5.095398
4,4.221878,2.108368,-5.494284,-4.295850,5.816855,2.786706,-7.993175,-7.454540,-4.028846,3.719992,...,6.070309,-6.167530,7.751654,-6.004386,7.264500,-9.219711,-9.710474,-8.983231,-0.622292,9.873538


In [4]:
estimator = Concat(kmeans__n_clusters = 2)
estimator.fit(X = imvd)

/home/alberto/anaconda3/envs/imc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('fillmissingviews', FillMissingViews()),
                ('functiontransformer',
                 FunctionTransformer(func=<function ConcatenateViews.__new__.<locals>.<lambda> at 0x7f03fd23c1f0>)),
                ('normalizer', Normalizer()),
                ('kmeans', KMeans(n_clusters=2))])

In [5]:
pred = estimator.predict(X = imvd)
pred

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [6]:
X = estimator[:-1].fit_transform(imvd)
print(X.shape)

(100, 160)


In [7]:
InternalMetrics(X = imvd, estimator= estimator).compute(labels_pred= pred)

{'Dunn': 0.5678946862868822,
 'Silhouette': 0.3154929975732828,
 'Connectivity': 3.186111111111111}

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    stability = StabilityMetrics(X = imvd, estimator = estimator, n_times = 3, feature_frac = 0.3, random_state=42)

In [9]:
stability.compute(pred = pred)

{'APN': [0.0, 0.0, [0.0, 0.0, 0.0]],
 'AD': [0.01, 0.0, [0.01, 0.01, 0.01]],
 'ADM': [0.01, 0.0, [0.01, 0.01, 0.01]],
 'FOM': [0.07985235925365985,
  1.1598434309156995e-06,
  [0.07985317938681497, 0.07985071898734962, 0.07985317938681497]]}

In [10]:
apn = stability.apn(pred = pred)
ad = stability.ad(pred = pred)
adm = stability.adm(pred = pred)
fom = stability.fom(pred = pred)
apn, ad, adm, fom

([0.0, 0.0, [0.0, 0.0, 0.0]],
 [0.01, 0.0, [0.01, 0.01, 0.01]],
 [0.01, 0.0, [0.01, 0.01, 0.01]],
 [0.07985235925365985,
  1.1598434309156995e-06,
  [0.07985317938681497, 0.07985071898734962, 0.07985317938681497]])

In [11]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    external_validation = ExternalValidation(X = imvd, sample_views = sample_views, estimator = estimator, n_times = 10, sample_frac = 0.5, random_state=42)
external_validation.compute_stability()

(AMI            -0.009100
 ARI             0.002206
 Completeness    0.009914
 FMI             0.594408
 dtype: float64,
 AMI             0.011341
 ARI             0.037126
 Completeness    0.011981
 FMI             0.057075
 dtype: float64,
         AMI       ARI  Completeness       FMI
 0 -0.016268 -0.018983      0.000087  0.527072
 1 -0.019399  0.008157      0.001539  0.641629
 2 -0.000010  0.005747      0.015175  0.502865
 3 -0.010148 -0.023392      0.006584  0.529150
 4  0.015737 -0.074306      0.038667  0.612672
 5  0.000049  0.071805      0.020103  0.664636
 6 -0.011210  0.017254      0.007352  0.590562
 7 -0.019399  0.008157      0.001539  0.641629
 8 -0.019140  0.010368      0.001773  0.643305
 9 -0.011210  0.017254      0.006320  0.590562)

In [12]:
estimator = NonnegativeMatrixFactorization()
estimator.fit(X = imvd)

/home/alberto/anaconda3/envs/imc/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/home/alberto/anaconda3/envs/imc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('fillmissingviews', FillMissingViews()),
                ('functiontransformer-1',
                 FunctionTransformer(func=<function ConvertToPositive.__new__.<locals>.<lambda> at 0x7f03fce99990>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function ConcatenateViews.__new__.<locals>.<lambda> at 0x7f03fce9a320>)),
                ('nmf', NMF()), ('kmeans', KMeans())])

In [107]:
from scipy.stats import chi2_contingency
from joblib import Parallel, delayed
from sklearn.model_selection import ShuffleSplit
import joblib
import pandas as pd


class LabelEvaluation():
    
    def __init__(self, clusters):
        self.clusters = clusters
    
    def compute(self, label, statistic : str, permutation : bool = True, sample_frac = .8, n_times = int(1e4), random_state = None, n_jobs = 1, verbose = 0):
        if statistic == "chi2":
            test = lambda x,y: chi2_contingency(pd.crosstab(x+1, y+1))[1]
        
        permutation_scores = joblib.Parallel(n_jobs = n_jobs, verbose = verbose)(
joblib.delayed(test)(self.clusters.iloc[idx], label.iloc[idx]) for idx,_ in ShuffleSplit(n_splits = n_times, train_size = sample_frac, random_state = random_state).split(self.clusters))

            
        return permutation_scores    


In [108]:
LabelEvaluation(clusters= external_validation.val_preds[0]).compute(label = external_validation.val_labels[0], statistic = "chi2", n_jobs = -1)

[0.9460511210776997,
 1.0,
 0.4566759804975824,
 1.0,
 1.0,
 0.7801029136015583,
 1.0,
 1.0,
 1.0,
 0.3305490721984653,
 1.0,
 1.0,
 1.0,
 0.6629719512744012,
 1.0,
 0.5364566513297457,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4573923689532905,
 1.0,
 1.0,
 0.6125971528470967,
 1.0,
 1.0,
 0.4573923689532905,
 0.6629719512744012,
 0.8447156864516001,
 1.0,
 1.0,
 0.7810104921096594,
 0.7810104921096594,
 1.0,
 0.6766757397819312,
 1.0,
 0.8428843334609427,
 0.8640789330442749,
 0.8362249774203172,
 1.0,
 1.0,
 0.4573923689532905,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7388826803635273,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8640789330442749,
 1.0,
 1.0,
 1.0,
 0.6629719512744012,
 0.6012932846160325,
 0.8281962042260473,
 0.6629719512744012,
 1.0,
 1.0,
 0.8640789330442749,
 1.0,
 0.9717703713517798,
 0.9717703713517798,
 1.0,
 1.0,
 0.9460511210776997,
 0.9460511210776997,
 1.0,
 0.9717703713517798,
 1.0,
 1.0,
 1.0,
 0.6584923203753067,
 0.9717703713517798,
 1.0,
 0.9327961245393529,
 1.0,
 1.0,
 

In [88]:
import joblib
from sklearn.model_selection import ShuffleSplit

n_splits, sample_frac, n_jobs, verbose, random_state = 10, .8, 2, 0, None
clusters = external_validation.val_preds[0]
label = external_validation.val_labels[0]

permutation_scores = joblib.Parallel(n_jobs=n_jobs, verbose=verbose)(
    joblib.delayed()(clusters.iloc[idx], label.iloc[idx]) for idx,_ in ShuffleSplit(n_splits = n_splits, train_size = sample_frac, random_state = random_state).split(clusters))
permutation_scores

[1.0,
 1.0,
 0.7937160633795738,
 1.0,
 1.0,
 1.0,
 0.8362249774203172,
 0.6475630032890333,
 0.62200788957262,
 1.0]

In [ ]:
external_validation.train_pred[0].count()

In [ ]:
x

In [77]:
from scipy.stats import chisquare, permutation_test, chi2_contingency
import numpy as np
import pandas as pd

x = external_validation.val_preds[0] +1
y = external_validation.val_labels[0] +1

result = chi2_contingency(pd.crosstab(x,y))
result

Chi2ContingencyResult(statistic=0.0, pvalue=1.0, dof=1, expected_freq=array([[21.12, 11.88],
       [10.88,  6.12]]))

In [55]:
result

Chi2ContingencyResult(statistic=0.0, pvalue=1.0, dof=1, expected_freq=array([[21.12, 11.88],
       [10.88,  6.12]]))

In [82]:
chi2_contingency(ct, axis = -1)

TypeError: chi2_contingency() got an unexpected keyword argument 'axis'

In [80]:
res = permutation_test(ct, lambda x: chi2_contingency(x)[0])
res.pvalue

AxisError: `axis` is out of bounds for array of dimension 0

In [69]:
from sklearn.feature_selection import chi2

chi2(x.to_frame(), y)

(array([0.00053419]), array([0.98156053]))